In [25]:
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csc_matrix, bmat, vstack, save_npz, csc_matrix, load_npz
import numpy as np

import time
import pickle
import subprocess

In [26]:
DATA = 'data/'

# Read data
with open(DATA + 'statements', 'r') as f:
    statements = f.read().splitlines()
    stmts_names = [st.split(',')[0][len('fof('):] for st in statements]
    
with open(DATA + 'chronology', 'r') as f:
    chronology = f.read().splitlines()
    
with open(DATA + 'features', 'r') as f:
    features = f.read().splitlines()
    features = {f.split(':')[0] : f.split(':')[1].split(', ') for f in features}
    
with open(DATA + 'dependencies_train', 'r') as f:
    train_dep_lines = f.read().splitlines()
    train_dep = {}
    for f in train_dep_lines:
        stmt = f.split(':')[0]
        prms = f.split(':')[1].split(' ')
        if stmt in train_dep.keys():
            train_dep[stmt] += prms
        else:
            train_dep[stmt] = prms
    
with open(DATA + 'conjectures_test', 'r') as f:
    conj_test = f.read().splitlines()
    
map_chrono = {f : i for i, f in enumerate(chronology)}

flatten = lambda t: [item for sublist in t for item in sublist]
land = lambda l1, l2: [a and b for a, b in zip(l1, l2)]
lor = lambda l1, l2: [a or b for a, b in zip(l1, l2)]
lxor = lambda l1, l2: [a != b for a, b in zip(l1, l2)]

# Prepare features sparse vectors
fts = list(set(flatten(list(features.values()))))
map_fts = {f : i for i, f in enumerate(fts)}

# Now I will convert statements to vectors
stmts = {st: [int(f in features[st]) for f in fts] for st in stmts_names}

def extract_feature (conj1, conj2):
    return land(conj1, conj2) + lor(conj1, conj2) + lxor(conj1, conj2)

In [27]:
def run_proof (conj, premises):
    with open('premises.txt', 'w') as p_file:
        for p in premises:
            p_file.write(p)
            p_file.write('\n')
    subprocess.run("export EPROVER=~/EProver/Ecompiledlinux/E/PROVER/eprover", shell = True)
    res = subprocess.run("python3 run_E_prover.py --conjecture " + conj_name
                   + ' --premises premises.txt'
                   + ' --statements data/statements', shell = True, capture_output=True)
    found = 'FOUND' in str(res.stdout) and 'NOT FOUND' not in str(res.stdout)
    print(res.stdout)
    return int(found)

def score_and_prove(forest, conj):
    scores = forest.predict_proba([extract_feature(conj, stmts[dep_name]) for dep_name in chronology[:map_chrono[conj_name]]])

    premises = []
    for i, s in enumerate(scores):
        if (s[1] > 0.9):
            premises.append(chronology[:map_chrono[conj_name]][i])

    return run_proof(conj, premises)

In [28]:
len(trainY)

383851

In [16]:
r, c = trainX.nonzero()

In [19]:
r.sort()
r

array([     0,      0,      0, ..., 383850, 383850, 383850], dtype=int32)

In [29]:
trainX3 = load_npz('full_train_data/fullTrainX200Three.npz')
with open('full_train_data/fullTrainY200Three', 'rb') as ty:
    trainY3 = pickle.load(ty)

In [30]:
trainX3

<396948x30723 sparse matrix of type '<class 'numpy.float64'>'
	with 32373154 stored elements in COOrdinate format>

In [ ]:
forest = RandomForestClassifier(random_state=2137)
forest.fit(trainX3, trainY3)

In [34]:
conj_name = conj_test[2]
#conj_name = list(train_dep.keys())[100]
conj = stmts[conj_name]
scores = forest.predict_proba([extract_feature(conj, stmts[dep_name]) for dep_name in chronology[:map_chrono[conj_name]]])

In [40]:
def max_score(scores):
    m = 0
    for s in scores:
        if s[1] > m:
            m = s[1]
    return m
max_score(scores)

0.4339047221147163

In [86]:
for s in scores:
    if s[1] > 0.6:
        print("hura")

In [43]:
sum(stmts[chronology[:map_chrono[conj_name]][100]])
map_chrono[conj_name]

232

In [42]:
len(scores)

232

In [16]:
proven = 0
for conj_name in list(train_dep.keys()):
    print(conj_name)
    proven += score_and_prove(forest, stmts[conj_name])
print("PROVED: ", 100 * proven / len(conj_test), "% of test set")

t3_xboole_0
b'Proof of conjecture t3_xboole_0 FOUND. Output in file t3_xboole_0.E_output\n'
t7_xboole_0
b'Proof of conjecture t7_xboole_0 FOUND. Output in file t7_xboole_0.E_output\n'
t6_xboole_1
b'Proof of conjecture t6_xboole_1 FOUND. Output in file t6_xboole_1.E_output\n'
t9_xboole_1
b'Proof of conjecture t9_xboole_1 FOUND. Output in file t9_xboole_1.E_output\n'
t10_xboole_1
b'Proof of conjecture t10_xboole_1 FOUND. Output in file t10_xboole_1.E_output\n'
t11_xboole_1
b'Proof of conjecture t11_xboole_1 FOUND. Output in file t11_xboole_1.E_output\n'
t12_xboole_1
b'Proof of conjecture t12_xboole_1 FOUND. Output in file t12_xboole_1.E_output\n'
t15_xboole_1
b'Proof of conjecture t15_xboole_1 FOUND. Output in file t15_xboole_1.E_output\n'
t17_xboole_1
b'Proof of conjecture t17_xboole_1 FOUND. Output in file t17_xboole_1.E_output\n'
t18_xboole_1
b'Proof of conjecture t18_xboole_1 FOUND. Output in file t18_xboole_1.E_output\n'
t20_xboole_1
b'Proof of conjecture t20_xboole_1 FOUND. Output 

KeyboardInterrupt: 